In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import style
style.use('seaborn-white')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import tensorflow as tf 
from tensorflow import keras

In [2]:
# put your local path in here
df = pd.read_excel(r"C:\Users\ymehr\Downloads\ValidData-MitC2022data - SalesPopulation.xlsx",sheet_name='filtered')

In [2]:
valid_df = pd.read_excel('other_files/valid_data_with_year.xlsx')

In [3]:
valid_df.head()

,Unnamed: 0,NBHD,PID,Situs,PrpCls,PrUse,CDU,Qual,YrRD,Remd,...,HSALandValue,TotAcres,V/I,Inst#,SaleDate,Price,SaleValidity,Sum801TLA,Raw Year,Relative year
0,0,7HST242,09-13-434-003-000,2525 RIDGE RD,R1,1,4.0,6.0,NaN,NaN,...,NaN,1.30,I,2016017053,2016-05-05,2122450.0,1,12026,2016,0
1,1,7HSE361,16-11-404-012-000,5330 S 93RD PL,R1,1,4.0,6.0,NaN,NaN,...,NaN,0.78,I,2022024921,2022-06-02,1950000.0,1,11047,2022,6
2,2,7HSE361,16-11-404-013-000,5410 S 93RD PL,R1,1,4.0,5.0,NaN,NaN,...,NaN,0.73,I,2021068025,2021-11-11,1875000.0,1,8816,2021,5
3,3,7AEA310,23-31-416-001-000,11737 SEABISCUIT LN,R1,12,4.0,5.0,NaN,NaN,...,1933-05-06 00:00:00,3.03,I,2022028838,2022-06-25,1850000.0,1,6935,2022,6
4,4,7HST242,09-13-402-005-000,2421 RIDGE RD,R1,1,5.0,5.0,2022.0,6.0,...,NaN,0.58,I,2022024045,2022-06-01,1800000.0,1,9936,2022,6


In [4]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34125 entries, 0 to 34124
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     34125 non-null  int64         
 1   NBHD           34125 non-null  object        
 2   PID            34125 non-null  object        
 3   Situs          34125 non-null  object        
 4   PrpCls         34125 non-null  object        
 5   PrUse          34125 non-null  int64         
 6   CDU            34116 non-null  float64       
 7   Qual           34116 non-null  float64       
 8   YrRD           3698 non-null   float64       
 9   Remd           3790 non-null   float64       
 10  Imp            34116 non-null  object        
 11  TLA            34123 non-null  float64       
 12  YrBlt          34116 non-null  float64       
 13  GarCap         31197 non-null  float64       
 14  Bedrm          34115 non-null  float64       
 15  FND            3411

In [19]:
num_df = valid_df.select_dtypes(include=np.number)
num_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34125 entries, 0 to 34124
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     34125 non-null  int64  
 1   PrUse          34125 non-null  int64  
 2   CDU            34116 non-null  float64
 3   Qual           34116 non-null  float64
 4   YrRD           3698 non-null   float64
 5   Remd           3790 non-null   float64
 6   TLA            34123 non-null  float64
 7   YrBlt          34116 non-null  float64
 8   GarCap         31197 non-null  float64
 9   Bedrm          34115 non-null  float64
 10  FND            34111 non-null  float64
 11  801Units       29842 non-null  float64
 12  802Units       3786 non-null   float64
 13  803Units       22494 non-null  float64
 14  FP             14955 non-null  float64
 15  FixCt          34118 non-null  float64
 16  Pool           304 non-null    float64
 17  Acres          1176 non-null   float64
 18  LandVa

In [22]:
num_df['Pool'].isna().sum()

33821

In [26]:
useful_df = num_df[[col for col in num_df.columns if num_df[col].isna().sum() < 10000]]
useful_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34125 entries, 0 to 34124
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     34125 non-null  int64  
 1   PrUse          34125 non-null  int64  
 2   CDU            34116 non-null  float64
 3   Qual           34116 non-null  float64
 4   TLA            34123 non-null  float64
 5   YrBlt          34116 non-null  float64
 6   GarCap         31197 non-null  float64
 7   Bedrm          34115 non-null  float64
 8   FND            34111 non-null  float64
 9   801Units       29842 non-null  float64
 10  FixCt          34118 non-null  float64
 11  LandValue      34125 non-null  int64  
 12  TotAcres       33395 non-null  float64
 13  Inst#          34125 non-null  int64  
 14  Price          34125 non-null  float64
 15  SaleValidity   34125 non-null  int64  
 16  Sum801TLA      34125 non-null  int64  
 17  Raw Year       34125 non-null  int64  
 18  Relati

In [27]:
useful_df.dropna(how='any', inplace=True)
useful_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28043 entries, 0 to 34124
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     28043 non-null  int64  
 1   PrUse          28043 non-null  int64  
 2   CDU            28043 non-null  float64
 3   Qual           28043 non-null  float64
 4   TLA            28043 non-null  float64
 5   YrBlt          28043 non-null  float64
 6   GarCap         28043 non-null  float64
 7   Bedrm          28043 non-null  float64
 8   FND            28043 non-null  float64
 9   801Units       28043 non-null  float64
 10  FixCt          28043 non-null  float64
 11  LandValue      28043 non-null  int64  
 12  TotAcres       28043 non-null  float64
 13  Inst#          28043 non-null  int64  
 14  Price          28043 non-null  float64
 15  SaleValidity   28043 non-null  int64  
 16  Sum801TLA      28043 non-null  int64  
 17  Raw Year       28043 non-null  int64  
 18  Relati

<ipython-input-27-db96c5c00aea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_df.dropna(how='any', inplace=True)


In [29]:
useful_df.drop(columns=['Unnamed: 0','Raw Year'], inplace=True)

c:\Users\ymehr\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [30]:
useful_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28043 entries, 0 to 34124
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PrUse          28043 non-null  int64  
 1   CDU            28043 non-null  float64
 2   Qual           28043 non-null  float64
 3   TLA            28043 non-null  float64
 4   YrBlt          28043 non-null  float64
 5   GarCap         28043 non-null  float64
 6   Bedrm          28043 non-null  float64
 7   FND            28043 non-null  float64
 8   801Units       28043 non-null  float64
 9   FixCt          28043 non-null  float64
 10  LandValue      28043 non-null  int64  
 11  TotAcres       28043 non-null  float64
 12  Inst#          28043 non-null  int64  
 13  Price          28043 non-null  float64
 14  SaleValidity   28043 non-null  int64  
 15  Sum801TLA      28043 non-null  int64  
 16  Relative year  28043 non-null  int64  
dtypes: float64(11), int64(6)
memory usage: 3.9 MB


In [33]:
X = useful_df[[col for col in useful_df.columns if col != 'Price']]
Y = useful_df["Price"]
# Y = np.reshape(list(Y),(-1,1))
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)

In [59]:
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=[len(X.columns)], activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='linear')
    ])

model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

model.fit(x_train,y_train,epochs=10)

Epoch 1/10
658/658 [==============================] - 2s 2ms/step - loss: 56720160194560.0000 - accuracy: 0.0000e+00
Epoch 2/10
658/658 [==============================] - 2s 3ms/step - loss: 21036861440.0000 - accuracy: 0.0000e+00
Epoch 3/10
658/658 [==============================] - 2s 2ms/step - loss: 34035623936.0000 - accuracy: 0.0000e+00
Epoch 4/10
658/658 [==============================] - 2s 2ms/step - loss: 10140445048832.0000 - accuracy: 0.0000e+00
Epoch 5/10
658/658 [==============================] - 2s 2ms/step - loss: 43401003008.0000 - accuracy: 0.0000e+00
Epoch 6/10
658/658 [==============================] - 1s 2ms/step - loss: 209987174400.0000 - accuracy: 0.0000e+00
Epoch 7/10
658/658 [==============================] - 2s 2ms/step - loss: 261297995776.0000 - accuracy: 0.0000e+00
Epoch 8/10
658/658 [==============================] - 2s 2ms/step - loss: 214591176704.0000 - accuracy: 4.7547e-05
Epoch 9/10
658/658 [==============================] - 2s 3ms/step - loss: 19975

In [47]:
model.evaluate(x_test,y_test)

220/220 [==============================] - 1s 2ms/step - loss: 16794276864.0000 - accuracy: 0.0000e+00


[16794276864.0, 0.0]